# CIE Project A - Group 7
## ---> Data processing

### Imports

In [28]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sp
from sklearn import preprocessing ##importing for normalization

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler

plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True
plt.rcParams['legend.frameon'] = True
%matplotlib widget

# import tensorflow as tf
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix

#### Get the root directory and change to it

In [29]:
rootdir = os.getcwd()
os.chdir(rootdir)

### Variable Assignment

In [30]:
gyrFile = 'Gyroscope'
accFile = 'Accelerometer'

normal = 'Normal'
upstairs = 'Upstairs'
downstairs = 'Downstairs'

smartphone = [1, 3]

s1 = 'Smartphone1'
s3 = 'Smartphone3'

s = [s1, s3]

gaitpos = [normal, upstairs, downstairs]

### Data Parsing

#### Define variables for raw data

In [31]:
count = 0

# Naming convention - gyr(oscope)_s(martphone_location)) and Same for acceleraoemeter
# All the data is stored in a list of dataframes. So, gyr{gaitpos}_s{martphone_location}[0, 1, 2, .....] gives the data as sorted by name in the file directory.
# gyrNormal_s1[0] gives the gyroscope data of the subject180_Normal sample 01 in the normal gait position.
# gyrNormal_s1[1] gives the gyroscope data of the subject180_Normal sample 02 in the normal gait position.
# 
# gyrNormal_s[2] gives the gyroscope data of the subject181_Normal sample 01 in the normal gait position.
# 
# and so on. 

# defining arrays for gyroscope data
gyrNormal_s1 = []; gyrUpstairs_s1 = []; gyrDownstairs_s1 = []
gyrNormal_s3 = []; gyrUpstairs_s3 = []; gyrDownstairs_s3 = []

#defining arrays for accelerometer data
accNormal_s1 = []; accUpstairs_s1 = []; accDownstairs_s1 = []
accNormal_s3 = []; accUpstairs_s3 = []; accDownstairs_s3 = []

#### Parsing gyroscope and accelerometer data at all gait positions (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [32]:
for subdir, dirs, files in os.walk(rootdir):
    for foldername in s:
        for gait in gaitpos:    
        
            gyr = eval(f'gyr{gait}_s{foldername[10]}')
            acc = eval(f'acc{gait}_s{foldername[10]}')
            
            if foldername.lower() in subdir.lower():
                if gait.lower() in subdir.lower():
    
                    for file in files:

                        if gyrFile.lower() in file.lower():
                            count = count + 1
                            # print(f'#{count} -- {os.path.join(subdir, file)}')

                            with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                                gyrData = pd.read_csv(f)
                                # Assemble the data into a tensor
                                gyr.append(gyrData)

                        if accFile in file:
                            count = count + 1
                            # print(f'#{count} -- {os.path.join(subdir, file)}')

                            with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                                accData = pd.read_csv(f)
                                # Assemble the data into a tensor
                                acc.append(accData)

#### Check if the data is collected correctly after parsing

In [33]:
# getting the data shape for reference
for j in gaitpos:
    for i in smartphone:
            gyrl = eval(f'gyr{j}_s{i}')
            accl = eval(f'acc{j}_s{i}')

            print(f'Shape of gyr{j}_s{i} is ------ {np.asarray((gyrl), dtype = list).shape}')
            print(f'Shape of acc{j}_s{i} is ------ {np.asarray((accl), dtype = list).shape} \n')

Shape of gyrNormal_s1 is ------ (8,)
Shape of accNormal_s1 is ------ (8,) 

Shape of gyrNormal_s3 is ------ (8,)
Shape of accNormal_s3 is ------ (8,) 

Shape of gyrUpstairs_s1 is ------ (8,)
Shape of accUpstairs_s1 is ------ (8,) 

Shape of gyrUpstairs_s3 is ------ (8,)
Shape of accUpstairs_s3 is ------ (8,) 

Shape of gyrDownstairs_s1 is ------ (8,)
Shape of accDownstairs_s1 is ------ (8,) 

Shape of gyrDownstairs_s3 is ------ (8,)
Shape of accDownstairs_s3 is ------ (8,) 



### Data Validation and Preprocessing

#### -- No need to run -- Frequency check of all the data before filtering

In [34]:
freqGyr = []
freqAcc = []

# Loop through the variables and check if all the data is collected
for j in gaitpos:
    for i in smartphone:
        # Calculating gyroscope frequency before preprocessing
        gyrl = eval(f'gyr{j}_s{i}')
        ng = len(gyrl)

        # loop to calculate the frequency over all the data
        for s in range(0, ng):
            time = np.asarray(gyrl[s].iloc[:,0])

            freqG = np.round(len(time)/(time[-1]))
            freqGyr.append(freqG)
        
        # Calculating accelerometer frequency before preprocessing
        accl = eval(f'acc{j}_s{i}')
        na = len(accl)

        # loop to calculate the frequency over all the data
        for s in range(0, na):
            time = np.asarray(accl[s].iloc[:,0])

            freqA = np.round(len(time)/(time[-1]))
            freqAcc.append(freqA)

for i in range(0, len(freqGyr)):
    if freqGyr[i] < 10:
        print(f'Position of the frequency issue is at freqGyr:{i} and the frequency is {freqGyr[i]}')
for j in range(0, len(freqAcc)):
    if freqAcc[j] < 10:
        print(f'Position of the frequency issue is at freqAcc:{j} and the frequency is {freqAcc[j]}')

### Filter Data

#### Lowpass filter (Butterworth) function

In [35]:
#Filter the data to remove noise

# inputs for the function is the data, the sampling frequency, the cutoff frequency.
def dataFilter(dataInput, fs=200, fc=5, order = 2):
    Filtrd = np.zeros(dataInput.shape)
    w = fc / (fs * 0.5)
    b, a = sp.butter(fc, w, 'low')
    
    #Filter the X, Y and Z axis of the input data
    for i in range(0, 3):
        Filtrd[:,i] = sp.filtfilt(b, a, dataInput.iloc[:,i+1])

    return Filtrd

##### -- Different filter check -- Chebyshev filter

In [36]:
'''
# chebyshev filter for the accelerometer data
def chebyFilter(dataInput, fs=200, fc=5, order = 2):
    Filtrd = np.zeros(dataInput.shape)
    w = fc / (fs * 0.5)
    b, a = sp.cheby2(order, 20, w, 'low')
    
    #Filter the X, Y and Z axis of the input data
    for i in range(0, 3):
        Filtrd[:,i] = sp.filtfilt(b,a, dataInput.iloc[:,i+1])
    
    return Filtrd
'''

"\n# chebyshev filter for the accelerometer data\ndef chebyFilter(dataInput, fs=200, fc=5, order = 2):\n    Filtrd = np.zeros(dataInput.shape)\n    w = fc / (fs * 0.5)\n    b, a = sp.cheby2(order, 20, w, 'low')\n    \n    #Filter the X, Y and Z axis of the input data\n    for i in range(0, 3):\n        Filtrd[:,i] = sp.filtfilt(b,a, dataInput.iloc[:,i+1])\n    \n    return Filtrd\n"

#### -- Test -- the dataFilter function and its dependency on the frequency input

In [37]:
'''gyrF = []
accF = []
ng = len(gyrNormal_s1)
na = len(accNormal_s1)

for i in range(0, ng):
    timeg = np.asarray(gyrNormal_s1[i].iloc[:,0])
    freqG = np.round(len(timeg)/(timeg[-1]))
    gyrF.append(freqG)

for j in range(0, na):
    timea = np.asarray(accNormal_s1[j].iloc[:,0])
    freqA = np.round(len(timea)/(timea[-1]))
    accF.append(freqA)

gyrFilt = []
accFilt = []

for rg in range(0, ng):
    gf = dataFilter(gyrNormal_s1[rg], fs = 100)
    gyrFilt.append(gf)

for ra in range(0, na):
    af = dataFilter(accNormal_s1[ra], fs = 100)
    accFilt.append(af)

t1 = np.asarray(gyrNormal_s1[0].iloc[:,0])
f1 = np.round(len(t1)/(t1[-1]))

xyz = dataFilter(gyrNormal_s1[0], fs = f1)

plt.close('all')
plt.plot(gyrNormal_s1[0].iloc[:,1], 'r')
plt.plot(xyz[:,0], 'b')
plt.show()'''

"gyrF = []\naccF = []\nng = len(gyrNormal_s1)\nna = len(accNormal_s1)\n\nfor i in range(0, ng):\n    timeg = np.asarray(gyrNormal_s1[i].iloc[:,0])\n    freqG = np.round(len(timeg)/(timeg[-1]))\n    gyrF.append(freqG)\n\nfor j in range(0, na):\n    timea = np.asarray(accNormal_s1[j].iloc[:,0])\n    freqA = np.round(len(timea)/(timea[-1]))\n    accF.append(freqA)\n\ngyrFilt = []\naccFilt = []\n\nfor rg in range(0, ng):\n    gf = dataFilter(gyrNormal_s1[rg], fs = 100)\n    gyrFilt.append(gf)\n\nfor ra in range(0, na):\n    af = dataFilter(accNormal_s1[ra], fs = 100)\n    accFilt.append(af)\n\nt1 = np.asarray(gyrNormal_s1[0].iloc[:,0])\nf1 = np.round(len(t1)/(t1[-1]))\n\nxyz = dataFilter(gyrNormal_s1[0], fs = f1)\n\nplt.close('all')\nplt.plot(gyrNormal_s1[0].iloc[:,1], 'r')\nplt.plot(xyz[:,0], 'b')\nplt.show()"

#### Filtering all the data

In [38]:
# defining arrays for filtered gyroscope data
gyrFilteredNormal_s1 = []; gyrFilteredUpstairs_s1 = []; gyrFilteredDownstairs_s1 = []
gyrFilteredDownstairs_s3 = []; gyrFilteredUpstairs_s3 = []; gyrFilteredNormal_s3 = []

# defining arrays for filtered accelerometer data
accFilteredNormal_s1 = []; accFilteredUpstairs_s1 = []; accFilteredDownstairs_s1 = []
accFilteredNormal_s3 = []; accFilteredUpstairs_s3 = []; accFilteredDownstairs_s3 = []

for j in gaitpos:
    for i in smartphone:
            # variables of raw the data
            gyrl = eval(f'gyr{j}_s{i}')
            accl = eval(f'acc{j}_s{i}')
            
            # variables of filtered the data
            gyrF = eval(f'gyrFiltered{j}_s{i}')
            accF = eval(f'accFiltered{j}_s{i}')

            # getting the shape of the gyroscope data (first integer)
            ng = len(gyrl)

            # Getting the shape of the accelerometer data (first integer)
            na = len(accl)


            # Filter all the data
            for rg in range(0, ng):
                timeg = np.asarray(gyrl[rg].iloc[:,0])
                fG = np.round(len(timeg)/(timeg[-1]))

                gf = dataFilter(gyrl[rg], fs = fG)
                gyrF.append(gf)

            for ra in range(0, na):
                timea = np.asarray(accl[ra].iloc[:,0])
                fA = np.round(len(timea)/(timea[-1]))

                af = dataFilter(accl[ra], fs = fA)
                accF.append(af)

#### Check if all the data is collected correctly after filtering

In [39]:
# getting the data shape for reference
for j in gaitpos:
    for i in smartphone:
            gyrl = eval(f'gyrFiltered{j}_s{i}')
            accl = eval(f'accFiltered{j}_s{i}')

            print(f'Shape of gyrFiltered{j}_s{i} is ------ {np.asarray((gyrl), dtype = list).shape}')
            print(f'Shape of accFiltered{j}_s{i} is ------ {np.asarray((accl), dtype = list).shape} \n')

Shape of gyrFilteredNormal_s1 is ------ (8,)
Shape of accFilteredNormal_s1 is ------ (8,) 

Shape of gyrFilteredNormal_s3 is ------ (8,)
Shape of accFilteredNormal_s3 is ------ (8,) 

Shape of gyrFilteredUpstairs_s1 is ------ (8,)
Shape of accFilteredUpstairs_s1 is ------ (8,) 

Shape of gyrFilteredUpstairs_s3 is ------ (8,)
Shape of accFilteredUpstairs_s3 is ------ (8,) 

Shape of gyrFilteredDownstairs_s1 is ------ (8,)
Shape of accFilteredDownstairs_s1 is ------ (8,) 

Shape of gyrFilteredDownstairs_s3 is ------ (8,)
Shape of accFilteredDownstairs_s3 is ------ (8,) 



### Cut data

#### Cut data function

In [40]:
def cutData(gyrFilt, accFilt, freq):
    acc_abs = np.linalg.norm(accFilt, axis=1)
    
    ht = np.round(np.mean(acc_abs[int(0.3*len(acc_abs)):int(0.7*len(acc_abs))])).astype(int)

    peaks, _ = sp.find_peaks(acc_abs, height = -10, distance=freq*0.5)
    diff_peaks = np.diff(peaks)
    gap1 = np.argmax(diff_peaks[:int(0.4*(len(diff_peaks)))])
    gap2 = np.argmax(diff_peaks[-int(0.4*len(diff_peaks)):])
    
    gap2 = int(np.shape(diff_peaks)[0] - int(0.4*len(diff_peaks)) + gap2)

    gyr_cut = gyrFilt[peaks[gap1+1]:peaks[gap2], :]
    acc_cut = accFilt[peaks[gap1+1]:peaks[gap2], :]

    return gyr_cut, acc_cut

##### -- check -- Cut data function

In [41]:
'''
gyrNormalCtted, accNormalCtted = [], []

for x in range(0, len(gyrNormal_s3)):
    num = x
    
    nq = len(accNormal_s3[num])
    tq = np.asarray(gyrNormal_s3[num].iloc[:,0])
    freq = np.round(len(tq)/(tq[-1]))
    # print(f'The frequency of the data is {freq}')

    acc_abs = np.linalg.norm(accFilteredNormal_s3[num], axis=1)

    # ht = np.round(np.mean(acc_abs[int(0.2*len(acc_abs)):int(0.7*len(acc_abs))])).astype(int)
    # ht = np.round(np.mean(acc_abs)).astype(int)
    # print(f'The average height of the data is {ht}')
    peaks, _ = sp.find_peaks(acc_abs, height = 0, distance=freq*0.5)
    # print(f'The number of peaks is {(peaks)}')

    diff_peaks = np.diff(peaks)
    # print(f'The difference of peaks is {(diff_peaks)}')

    gap1 = np.argmax(diff_peaks[:20])
    # print(f'The first gap is {gap1}')

    gap2 = np.argmax(diff_peaks[-10:])
    gap2 = int(np.shape(diff_peaks)[0] - 10 + gap2)
    # print(f'The second gap is {gap2}')

    gyr_cut = gyrFilteredNormal_s3[num][peaks[gap1+1]:peaks[gap2], :]
    acc_cut = accFilteredNormal_s3[num][peaks[gap1+1]:peaks[gap2], :]

    gyrNormalCtted.append(gyr_cut)
    accNormalCtted.append(acc_cut)
'''

"\ngyrNormalCtted, accNormalCtted = [], []\n\nfor x in range(0, len(gyrNormal_s3)):\n    num = x\n    \n    nq = len(accNormal_s3[num])\n    tq = np.asarray(gyrNormal_s3[num].iloc[:,0])\n    freq = np.round(len(tq)/(tq[-1]))\n    # print(f'The frequency of the data is {freq}')\n\n    acc_abs = np.linalg.norm(accFilteredNormal_s3[num], axis=1)\n\n    # ht = np.round(np.mean(acc_abs[int(0.2*len(acc_abs)):int(0.7*len(acc_abs))])).astype(int)\n    # ht = np.round(np.mean(acc_abs)).astype(int)\n    # print(f'The average height of the data is {ht}')\n    peaks, _ = sp.find_peaks(acc_abs, height = 0, distance=freq*0.5)\n    # print(f'The number of peaks is {(peaks)}')\n\n    diff_peaks = np.diff(peaks)\n    # print(f'The difference of peaks is {(diff_peaks)}')\n\n    gap1 = np.argmax(diff_peaks[:20])\n    # print(f'The first gap is {gap1}')\n\n    gap2 = np.argmax(diff_peaks[-10:])\n    gap2 = int(np.shape(diff_peaks)[0] - 10 + gap2)\n    # print(f'The second gap is {gap2}')\n\n    gyr_cut = 

##### -- Test -- Cut Data line by line

In [42]:
'''
num = 7

nq = len(accNormal_s3[num])
tq = np.asarray(gyrNormal_s3[num].iloc[:,0])
freq = np.round(len(tq)/(tq[-1]))
print(f'The frequency of the data is {freq}')

acc_abs = np.linalg.norm(accFilteredNormal_s3[num], axis=1)

ht = np.round(np.mean(acc_abs[int(0.2*len(acc_abs)):int(0.7*len(acc_abs))])).astype(int)
# ht = np.round(np.mean(acc_abs)).astype(int)
print(f'The average height of the data is {ht}')
peaks, _ = sp.find_peaks(acc_abs, height = 0, distance=freq*0.5)
print(f'The number of peaks is {(peaks)}')

diff_peaks = np.diff(peaks)
print(f'The difference of peaks is {(diff_peaks)}')

gap1 = np.argmax(diff_peaks[:int(0.1*(len(diff_peaks)))])
print(f'The first gap is {gap1}')

gap2 = np.argmax(diff_peaks[-int(0.1*len(diff_peaks)):])
gap2 = int(np.shape(diff_peaks)[0] - int(0.2*len(diff_peaks)) + gap2)
print(f'The second gap is {gap2}')

gyr_cut = gyrFilteredNormal_s3[num][peaks[gap1+1]:peaks[gap2], :]
acc_cut = accFilteredNormal_s3[num][peaks[gap1+1]:peaks[gap2], :]

plt.close()
# plt.plot(gyrFilteredNormal_s1[1][:,0], label='Raw Data', c = 'b')

plt.plot(acc_abs, label = 'Accelerometer Absolute Data', c = 'grey')
plt.plot(np.linspace(0, len(gyrNormal_s3[num])), np.full_like(np.linspace(0, len(gyrNormal_s3[num])), ht), label = 'Average Height', c = 'r')

plt.plot(peaks, acc_abs[peaks], 'x', label = 'Peaks', c = 'y')
plt.plot(np.linspace(0, len(gyrNormal_s3[num])), np.zeros_like(np.linspace(0, len(gyrNormal_s3[num]))), linestyle = "--", linewidth = 0.2, dashes=(5, 15), color="black")

plt.plot(peaks[gap1+1], acc_abs[peaks[gap1+1]], 'o', label = 'Gap 1', c = 'g')
plt.plot(peaks[gap2], acc_abs[peaks[gap2]], 'o', label = 'Gap 2', c = 'black')

plt.plot(gyr_cut[:,0], label = 'Gyr Cut', c='r')

plt.legend()
plt.grid()
plt.show()
'''

'\nnum = 7\n\nnq = len(accNormal_s3[num])\ntq = np.asarray(gyrNormal_s3[num].iloc[:,0])\nfreq = np.round(len(tq)/(tq[-1]))\nprint(f\'The frequency of the data is {freq}\')\n\nacc_abs = np.linalg.norm(accFilteredNormal_s3[num], axis=1)\n\nht = np.round(np.mean(acc_abs[int(0.2*len(acc_abs)):int(0.7*len(acc_abs))])).astype(int)\n# ht = np.round(np.mean(acc_abs)).astype(int)\nprint(f\'The average height of the data is {ht}\')\npeaks, _ = sp.find_peaks(acc_abs, height = 0, distance=freq*0.5)\nprint(f\'The number of peaks is {(peaks)}\')\n\ndiff_peaks = np.diff(peaks)\nprint(f\'The difference of peaks is {(diff_peaks)}\')\n\ngap1 = np.argmax(diff_peaks[:int(0.1*(len(diff_peaks)))])\nprint(f\'The first gap is {gap1}\')\n\ngap2 = np.argmax(diff_peaks[-int(0.1*len(diff_peaks)):])\ngap2 = int(np.shape(diff_peaks)[0] - int(0.2*len(diff_peaks)) + gap2)\nprint(f\'The second gap is {gap2}\')\n\ngyr_cut = gyrFilteredNormal_s3[num][peaks[gap1+1]:peaks[gap2], :]\nacc_cut = accFilteredNormal_s3[num][pea

#### Cut all the data

In [43]:
# defining arrays for cut gyroscope data
gyrCutNormal_s1 = []; gyrCutUpstairs_s1 = []; gyrCutDownstairs_s1 = []
gyrCutNormal_s3 = []; gyrCutUpstairs_s3 = []; gyrCutDownstairs_s3 = []

# defining arrays for cut accelerometer data
accCutNormal_s1 = []; accCutUpstairs_s1 = []; accCutDownstairs_s1 = []
accCutNormal_s3 = []; accCutUpstairs_s3 = []; accCutDownstairs_s3 = []

for j in gaitpos:
    for i in smartphone:
            # variables of raw the data
            gyrl = eval(f'gyr{j}_s{i}')
            accl = eval(f'acc{j}_s{i}')
            
            # variables of filtered the data
            gyrF = eval(f'gyrFiltered{j}_s{i}')
            accF = eval(f'accFiltered{j}_s{i}')

            # variables of cutted the data
            gyrC = eval(f'gyrCut{j}_s{i}')
            accC = eval(f'accCut{j}_s{i}')

            # getting the shape of the gyroscope and accelerometer data
            n = len(gyrF)

            # Filter all the data
            for r in range(0, n):
                
                tCut = np.asarray(gyrl[r].iloc[:,0])
                freqCut = np.round(len(tCut)/(tCut[-1]))
                # print(f'accFiltered{j}_s{i} - {[r]}')
                gf, aF = cutData(gyrF[r], accF[r], freqCut)

                gyrC.append(gf)
                accC.append(aF)

#### check if all the data is collected correctly after cutting

In [44]:
# getting the data shape for reference
for j in gaitpos:
    for i in smartphone:
            gyr = eval(f'gyrCut{j}_s{i}')
            acc = eval(f'accCut{j}_s{i}')

            print(f'Shape of gyrCut{j}_s{i} is ------ {np.asarray((gyr), dtype = list).shape}')
            print(f'Shape of accCut{j}_s{i} is ------ {np.asarray((acc), dtype = list).shape} \n')

Shape of gyrCutNormal_s1 is ------ (8,)
Shape of accCutNormal_s1 is ------ (8,) 

Shape of gyrCutNormal_s3 is ------ (8,)
Shape of accCutNormal_s3 is ------ (8,) 

Shape of gyrCutUpstairs_s1 is ------ (8,)
Shape of accCutUpstairs_s1 is ------ (8,) 

Shape of gyrCutUpstairs_s3 is ------ (8,)
Shape of accCutUpstairs_s3 is ------ (8,) 

Shape of gyrCutDownstairs_s1 is ------ (8,)
Shape of accCutDownstairs_s1 is ------ (8,) 

Shape of gyrCutDownstairs_s3 is ------ (8,)
Shape of accCutDownstairs_s3 is ------ (8,) 



#### Plot raw data, filtered data, and cut data

In [45]:
# pos = 'Normal', 'Upstairs', 'Downstairs'
pos = 'Normal'
# loc = 1, 3
loc = 3
# no = 0-7
no = 4
# axis = 0, 1, 2
axis = 0

plt.close()
plt.plot(eval(f'gyr{pos}_s{loc}')[no].iloc[:,(axis+1)], label='Raw Data', c = 'c')
plt.plot(eval(f'gyrFiltered{pos}_s{loc}')[no][:,axis], label='Filtered Data', c = 'b')
plt.plot(eval(f'gyrCut{pos}_s{loc}')[no][:,axis], label = 'Gyr Cutted Data', c = 'r')
# plt.plot(eval(f'gyrResample{pos}_s{loc}')[no][:,axis], label = 'Gyr Resampled Data', c = 'g')

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### -- Test -- Save all cut data to plots

In [46]:
'''
for j in gaitpos:
    for i in smartphone:
        # variables of raw the data
        gyrl = eval(f'gyr{j}_s{i}')
        accl = eval(f'acc{j}_s{i}')
        # variables of filtered the data
        gyrF = eval(f'gyrFiltered{j}_s{i}')
        accF = eval(f'accFiltered{j}_s{i}')
        # variables of cutted the data
        gyrC = eval(f'gyrCut{j}_s{i}')
        accC = eval(f'accCut{j}_s{i}')

        for rg in range(0, len(gyrF)):
            plt.close()
            plt.plot(gyrl[rg].iloc[:,1], label='Raw Gyroscope Data', c = 'c')
            # plt.plot(accl[rg].iloc[:,1], label='Raw Accelerometer Data', c = 'm')

            plt.plot(gyrF[rg][:,0], label='Filtered Gyroscope Data', c = 'b')
            # plt.plot(accF[rg][:,0], label='Filtered Accelerometer Data', c = 'g')

            plt.plot(gyrC[rg][:,0], label='Gyroscope Cut Data', c = 'y')
            # plt.plot(accC[rg][:,0], label = 'Accelerometer Cut Data', c = 'r')

            plt.legend()
            plt.grid()
            plt.ioff()
            # plt.savefig(rootdir+f'/2 Accelerometer_Pictures/acc{j}_s{i}_{rg}.png')
            plt.savefig(rootdir+f'/1 Gyroscope_Pictures/gyr{j}_s{i}_{rg}.png')
'''

"\nfor j in gaitpos:\n    for i in smartphone:\n        # variables of raw the data\n        gyrl = eval(f'gyr{j}_s{i}')\n        accl = eval(f'acc{j}_s{i}')\n        # variables of filtered the data\n        gyrF = eval(f'gyrFiltered{j}_s{i}')\n        accF = eval(f'accFiltered{j}_s{i}')\n        # variables of cutted the data\n        gyrC = eval(f'gyrCut{j}_s{i}')\n        accC = eval(f'accCut{j}_s{i}')\n\n        for rg in range(0, len(gyrF)):\n            plt.close()\n            plt.plot(gyrl[rg].iloc[:,1], label='Raw Gyroscope Data', c = 'c')\n            # plt.plot(accl[rg].iloc[:,1], label='Raw Accelerometer Data', c = 'm')\n\n            plt.plot(gyrF[rg][:,0], label='Filtered Gyroscope Data', c = 'b')\n            # plt.plot(accF[rg][:,0], label='Filtered Accelerometer Data', c = 'g')\n\n            plt.plot(gyrC[rg][:,0], label='Gyroscope Cut Data', c = 'y')\n            # plt.plot(accC[rg][:,0], label = 'Accelerometer Cut Data', c = 'r')\n\n            plt.legend()\n      

### Resample data

#### Resample all the data

In [47]:
# Create arrays for resampled data
gyrResampleNormal_s1 = []; gyrResampleUpstairs_s1 = []; gyrResampleDownstairs_s1 = []
gyrResampleNormal_s3 = []; gyrResampleUpstairs_s3 = []; gyrResampleDownstairs_s3 = []

accResampleNormal_s1 = []; accResampleUpstairs_s1 = []; accResampleDownstairs_s1 = []
accResampleNormal_s3 = []; accResampleUpstairs_s3 = []; accResampleDownstairs_s3 = []

size = 200

for j in gaitpos:
    for i in smartphone:
            gyrl = eval(f'gyrCut{j}_s{i}')
            accl = eval(f'accCut{j}_s{i}')

            gyrrs = eval(f'gyrResample{j}_s{i}')
            accrs = eval(f'accResample{j}_s{i}')

            for rg in range(0, len(gyrl)):
                g = sp.resample(gyrl[rg], size, axis = 0)
                a = sp.resample(accl[rg], size, axis = 0)
                
                gyrrs.append(g)
                accrs.append(a)

#### Change the data type to pd.DataFrame, add a column for label and drop that extra frckin column

In [48]:

for j in gaitpos:
    for i in smartphone:
            gyrl = eval(f'gyrResample{j}_s{i}')
            accl = eval(f'accResample{j}_s{i}')

            for x in range(0, len(gyrl)):
                g = pd.DataFrame(data = gyrl[x], columns=['X (rad/s)', 'Y (rad/s)', 'Z (rad/s)', 'Unnamed'])
                a = pd.DataFrame(data = accl[x], columns=['X (m/s^2)', 'Y (m/s^2)', 'Z (m/s^2)', 'Unnamed'])
                
                g.drop(g.columns[3], axis=1, inplace=True)
                a.drop(a.columns[3], axis=1, inplace=True)

                gyrl[x] = g
                accl[x] = a

                gyrl[x]['Labelg'] = j
                accl[x]['Labela'] = j

#### Check if all the data is collected properly after resampling

In [49]:
# getting the data shape for reference
for j in gaitpos:
    for i in smartphone:
            gyr = eval(f'gyrResample{j}_s{i}')
            acc = eval(f'accResample{j}_s{i}')

            print(f'Shape of gyrResample{j}_s{i} is ------ {np.asarray((gyr), dtype = list).shape}')
            print(f'Shape of accResample{j}_s{i} is ------ {np.asarray((acc), dtype = list).shape} \n')

Shape of gyrResampleNormal_s1 is ------ (8, 200, 4)
Shape of accResampleNormal_s1 is ------ (8, 200, 4) 

Shape of gyrResampleNormal_s3 is ------ (8, 200, 4)
Shape of accResampleNormal_s3 is ------ (8, 200, 4) 

Shape of gyrResampleUpstairs_s1 is ------ (8, 200, 4)
Shape of accResampleUpstairs_s1 is ------ (8, 200, 4) 

Shape of gyrResampleUpstairs_s3 is ------ (8, 200, 4)
Shape of accResampleUpstairs_s3 is ------ (8, 200, 4) 

Shape of gyrResampleDownstairs_s1 is ------ (8, 200, 4)
Shape of accResampleDownstairs_s1 is ------ (8, 200, 4) 

Shape of gyrResampleDownstairs_s3 is ------ (8, 200, 4)
Shape of accResampleDownstairs_s3 is ------ (8, 200, 4) 



#### Plot to compare the cut data, and resampled data

In [50]:
# pos = 'Normal', 'Upstairs', 'Downstairs'
pos = 'Normal'
# loc = 1, 3
loc = 3
# no = 0-7
no = 2
# axis = 0, 1, 2
axis = 0

plt.close()
fig, axs = plt.subplots(2,1)
fig.suptitle('Comparision of Resampled Data')

# axs[0].plot(eval(f'gyr{pos}_s{loc}')[no].iloc[:,(axis+1)], label='Raw Data', c = 'c')
# axs[0].plot(eval(f'gyrFiltered{pos}_s{loc}')[no][:,axis], label='Filtered Data', c = 'b')
axs[0].plot(eval(f'gyrCut{pos}_s{loc}')[no][:,axis], label = 'Gyr Cutted Data', c = 'r')
axs[0].x_lim = [0, 250]

axs[1].plot(eval(f'gyrResample{pos}_s{loc}')[no].iloc[:,axis], label = 'Gyr Resampled Data', c = 'g')

axs[0].legend()
axs[1].legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Create combined dataframes

#### Create dataframes combining all subjects into gyr_s1, acc_s1 and so on

In [51]:
# combine all the data into gyr_s1, acc_s1, gyr_s3, and acc_s3
gyr_s1 = pd.DataFrame(columns=['X (rad/s)', 'Y (rad/s)', 'Z (rad/s)'])
acc_s1 = pd.DataFrame(columns=['X (m/s^2)', 'Y (m/s^2)', 'Z (m/s^2)'])

gyr_s3 = pd.DataFrame(columns=['X (rad/s)', 'Y (rad/s)', 'Z (rad/s)'])
acc_s3 = pd.DataFrame(columns=['X (m/s^2)', 'Y (m/s^2)', 'Z (m/s^2)'])

for i in smartphone:    
    for j in gaitpos:
        gyr = eval(f'gyrResample{j}_s{i}')
        acc = eval(f'accResample{j}_s{i}')
        
        if i == 1:    
            for x in range(0, len(gyr)):
                gyr_s1 = pd.concat((gyr_s1, gyr[x]), ignore_index=True, axis = 0)
                acc_s1 = pd.concat((acc_s1, acc[x]), ignore_index=True, axis = 0)
        
        elif i == 3:
            for x in range(0, len(gyr)):
                gyr_s3 = pd.concat((gyr_s3, gyr[x]), ignore_index=True, axis = 0)
                acc_s3 = pd.concat((acc_s3, acc[x]), ignore_index=True, axis = 0)


#### create dataframes for each smartphone

In [52]:
# defining arrays for subject specific data
s1 = pd.DataFrame(columns = ['X (rad/s)', 'Y (rad/s)', 'Z (rad/s)', 'X (m/s^2)', 'Y (m/s^2)', 'Z (m/s^2)', 'Labela'])
s3 = pd.DataFrame(columns = ['X (rad/s)', 'Y (rad/s)', 'Z (rad/s)', 'X (m/s^2)', 'Y (m/s^2)', 'Z (m/s^2)', 'Labela'])

s1 = pd.concat((gyr_s1, acc_s1), axis = 1)
s3 = pd.concat((gyr_s3, acc_s3), axis = 1)

s1.drop(['Labelg'], axis=1, inplace=True)
s3.drop(['Labelg'], axis=1, inplace=True)

#### Normalize the data

In [73]:
s1_n = []; s3_n = []

for i in smartphone:
    ss = eval(f's{i}')
    if i == 1:
        s1_n = preprocessing.normalize(ss.iloc[:,:6],axis=1)
    elif i == 3:
        s3_n = preprocessing.normalize(ss.iloc[:,:6],axis=1)

In [78]:
sp = pd.DataFrame(data = s3_n, columns = ['X (rad/s)', 'Y (rad/s)', 'Z (rad/s)', 'X (m/s^2)', 'Y (m/s^2)', 'Z (m/s^2)'])

,X (rad/s),Y (rad/s),Z (rad/s),X (m/s^2),Y (m/s^2),Z (m/s^2)
0,0.012393,-0.008922,0.026738,0.209931,-0.954451,-0.209774
1,-0.014021,-0.052409,0.010301,0.064516,-0.995836,-0.033144
2,-0.002086,0.013525,-0.008867,0.108136,-0.982736,-0.149231
3,-0.005284,0.001213,0.007737,0.117182,-0.990964,-0.064567
4,-0.021032,0.015040,-0.003838,0.180601,-0.968281,-0.170679
...,...,...,...,...,...,...
4795,-0.212004,0.086500,-0.145837,0.034939,0.959156,-0.071438
4796,-0.231833,0.059371,-0.153696,-0.066483,0.956271,0.015209
4797,-0.243383,0.052094,-0.161133,-0.085256,0.950999,0.020471
4798,-0.201496,0.047888,-0.120297,-0.102265,0.962073,0.081185


In [74]:
plt.close()
fig, axs = plt.subplots(2,1)
fig.suptitle('Comparision of Resampled Data')
axs[0].plot(s3_n[:,0], label = 'X (m/s^2)', c = 'r')
axs[1].plot(s3.iloc[:,0], label = 'X (m/s^2)', c = 'g')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### PCA

#### run PCA on all the data

In [55]:
## Run PCA on the data
for i in smartphone:
    phone = eval(f's{i}')
    phone = phone.iloc[:,:6]

    scale = StandardScaler().fit_transform(phone) # use transpose to get the correct shape

    pca = PCA(n_components = 6)
    pca.fit(scale)
    phone = pca.transform(scale)

phone.shape

(4800, 6)

In [56]:
var = np.round(pca.explained_variance_ratio_, decimals=3)*100
labels = [f'PC{x}' for x in range(1, len(var)+1)]

pcaDf = pd.DataFrame(phone, columns = labels)

#### Check the scree plot to define the number of components to keep

In [61]:
plt.close()
plt.bar(x = range(1, len(var)+1), height = var, tick_label = labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('SCREE PLOT')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Check which axis data is clustering from the scatter plot

In [58]:
loadingScores = pd.Series(phone[:,0], index = np.linspace(0,4799,4800))
sortedLoadingScores = loadingScores.abs().sort_values(ascending=False)

topData = sortedLoadingScores[0:3].index.values

print('Top two principal components are -')
print(loadingScores[topData])

Top two principal components are -
0.0      -1.084827
3432.0   -1.084786
4716.0    1.084506
3413.0   -1.084356
2392.0   -1.084013
            ...   
3003.0    0.362877
2126.0   -0.362380
494.0     0.362280
3565.0   -0.362097
3.0      -0.361862
Length: 1829, dtype: float64


In [59]:
sortedLoadingScores

4601.0    5.111259
4602.0    4.221376
3527.0    3.831997
3526.0    3.822709
4603.0    3.802500
            ...   
3350.0    0.004513
1522.0    0.002967
745.0     0.001769
3884.0    0.001727
3286.0    0.000723
Length: 4800, dtype: float64

In [60]:
plt.close()
plt.scatter(pcaDf.PC1, pcaDf.PC2, cmap = 'viridis')

plt.title('Principal Component Analysis')
plt.xlabel(f'PC1 - {var[0]}%')
plt.ylabel(f'PC2 - {var[1]}%')
plt.show()

# for name in pcaDf.index:
#     plt.annotate(name, (pcaDf.PC1[name], pcaDf.PC2[name]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Output from PCA (normalize before input to AI)

## ---> AI Model